In [2]:
import cv2
from ultralytics import YOLO
import time
import paho.mqtt.publish as publish

In [ ]:
# Load YOLOv8 model
model = YOLO("yolov8n.pt")  # yolov8n = nano, fast

# ESP32-CAM stream URL
stream_url = "http://192.168.1.229:81/stream"  # Replace with your IP
cap = cv2.VideoCapture(stream_url)

def trigger_alert():
    UBIDOTS_BROKER = "industrial.api.ubidots.com"
    UBIDOTS_TOKEN = "BBUS-iYTamlBTLRQ8di2mUMohiW4ErEmBGf"
    DEVICE_LABEL = "hazard-node"
    VARIABLE_LABEL = "alert"

    topic = f"/v1.6/devices/{DEVICE_LABEL}/{VARIABLE_LABEL}"
    payload = "{\"value\":1}"  # you can send 0 to turn off

    publish.single(
        topic,
        payload=payload,
        hostname=UBIDOTS_BROKER,
        port=1883,
        auth={'username': UBIDOTS_TOKEN, 'password': ''},
    )
    print("[ALERT SENT] to Ubidots")


while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame")
        continue

    results = model(frame)
    boxes = results[0].boxes

    for box in boxes:
        cls = int(box.cls[0])
        label = model.names[cls]
        x1, y1, x2, y2 = box.xyxy[0]
        area = (x2 - x1) * (y2 - y1)

        # Check for large 'person' = close to camera
        if label == "person" and area > 50000:
            trigger_alert()

    # Show annotated frame
    annotated = results[0].plot()
    cv2.imshow("ESP32 YOLO Detection", annotated)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()



0: 480x640 1 person, 63.6ms
Speed: 2.4ms preprocess, 63.6ms inference, 1.0ms postprocess per image at shape (1, 3, 480, 640)
[ALERT] Object too close!

0: 480x640 1 person, 49.0ms
Speed: 1.4ms preprocess, 49.0ms inference, 0.7ms postprocess per image at shape (1, 3, 480, 640)
[ALERT] Object too close!

0: 480x640 (no detections), 42.9ms
Speed: 1.2ms preprocess, 42.9ms inference, 0.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 43.1ms
Speed: 1.3ms preprocess, 43.1ms inference, 0.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 40.8ms
Speed: 1.7ms preprocess, 40.8ms inference, 0.5ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 36.2ms
Speed: 1.9ms preprocess, 36.2ms inference, 0.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 39.6ms
Speed: 1.2ms preprocess, 39.6ms inference, 0.3ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 (no detections), 45

KeyboardInterrupt: 

: 

In [3]:
!pip install streamlit

  Using cached cachetools-5.5.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached watchdog-6.0.0-py3-none-win_amd64.whl.metadata (44 kB)
  Using cached GitPython-3.1.44-py3-none-any.whl.metadata (13 kB)
  Using cached jsonschema-4.23.0-py3-none-any.whl.metadata (7.9 kB)
  Using cached gitdb-4.0.12-py3-none-any.whl.metadata (1.2 kB)
  Using cached smmap-5.0.2-py3-none-any.whl.metadata (4.3 kB)
  Using cached jsonschema_specifications-2024.10.1-py3-none-any.whl.metadata (3.0 kB)
   ---------------------------------------- 0.0/9.8 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.8 MB 991.0 kB/s eta 0:00:10
    --------------------------------------- 0.2/9.8 MB 2.3 MB/s eta 0:00:05
   - -------------------------------------- 0.4/9.8 MB 3.2 MB/s eta 0:00:03
   - -------------------------------------- 0.4/9.8 MB 3.2 MB/s eta 0:00:03
   -- ------------------------------------- 0.5/9.8 MB 2.4 MB/s eta 0:00:04
   -- ------------------------------------- 0.6/9.8 MB 2.1 MB/s eta


[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\hp\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [ ]:
import streamlit as st
import cv2
import numpy as np
from ultralytics import YOLO
from PIL import Image
import paho.mqtt.publish as publish
from datetime import datetime

# === CONFIG ===
STREAM_URL = "http://192.168.1.229:81/stream"  # Replace with your ESP32-CAM IP
UBIDOTS_TOKEN = "BBUS-iYTamlBTLRQ8di2mUMohiW4ErEmBGf"           # Replace with your token
UBIDOTS_BROKER = "industrial.api.ubidots.com"
DEVICE_LABEL = "hazard-node"
VARIABLE_LABEL = "alert"

# === STREAMLIT UI ===
st.set_page_config(page_title="ESP32-CAM Hazard Detector", layout="wide")
st.title("📸 ESP32-CAM Object Proximity Alert")

confidence_thresh = st.sidebar.slider("Confidence Threshold", 0.1, 1.0, 0.5)
area_thresh = st.sidebar.slider("Bounding Box Area Threshold", 10000, 150000, 50000)

# Manual Trigger Button
if st.sidebar.button("🔔 Manually Trigger Alert"):
    topic = f"/v1.6/devices/{DEVICE_LABEL}/{VARIABLE_LABEL}"
    payload = "{\"value\":1}"
    publish.single(topic, payload, hostname=UBIDOTS_BROKER, port=1883,
                   auth={'username': UBIDOTS_TOKEN, 'password': ''})
    st.sidebar.success("Alert sent manually!")

# Load YOLOv8
model = YOLO("yolov8n.pt")

frame_holder = st.empty()
log_holder = st.empty()
logs = []

def trigger_alert():
    topic = f"/v1.6/devices/{DEVICE_LABEL}/{VARIABLE_LABEL}"
    payload = "{\"value\":1}"
    publish.single(topic, payload, hostname=UBIDOTS_BROKER, port=1883,
                   auth={'username': UBIDOTS_TOKEN, 'password': ''})
    st.toast("🚨 ALERT: Object Too Close!")

# Open camera stream
cap = cv2.VideoCapture(STREAM_URL)

while True:
    ret, frame = cap.read()
    if not ret:
        st.warning("⚠️ Failed to grab frame from camera.")
        break

    results = model(frame)[0]
    annotated = results.plot()

    for box in results.boxes:
        cls = int(box.cls[0])
        label = model.names[cls]
        x1, y1, x2, y2 = box.xyxy[0]
        area = (x2 - x1) * (y2 - y1)

        if label == "person" and area > area_thresh:
            trigger_alert()
            logs.append(f"[{datetime.now().strftime('%H:%M:%S')}] 🚨 {label.upper()} - Area: {int(area)}")

    frame_holder.image(cv2.cvtColor(annotated, cv2.COLOR_BGR2RGB), channels="RGB")
    if logs:
        log_holder.markdown("### 🚨 Detection Log\n" + "\n".join(logs[-10:]))

    if not st.runtime.exists(): break

cap.release()
